This notebook is used to generate connectivity and gradient used for the paper.

## Code Setup

In [1]:
# If True, assumes everything is running locally.
IS_LOCAL = False

# Path to main directory
REMOTE_PATH = "/content/gdrive/Shareddrives/Birds and CS/Data/CA-Final"
LOCAL_PATH = "/Users/luca/Library/CloudStorage/GoogleDrive-luca@ucsc.edu/Shared drives/Birds and CS/Data/CA-Final"
DATA_PATH = LOCAL_PATH if IS_LOCAL else REMOTE_PATH

In [2]:
import os
import sys

In [3]:
# Installs required packages
if not IS_LOCAL:
    !pip install git+https://github.com/ecoscape-earth/ecoscape-connectivity.git
    !pip install git+https://github.com/ecoscape-earth/ecoscape-utils.git

  Cloning https://github.com/ecoscape-earth/ecoscape-connectivity.git to /tmp/pip-req-build-2mh_m8e8
  Running command git clone --filter=blob:none --quiet https://github.com/ecoscape-earth/ecoscape-connectivity.git /tmp/pip-req-build-2mh_m8e8
  Resolved https://github.com/ecoscape-earth/ecoscape-connectivity.git to commit 23e3ef478fc073aa3da612ff8168ed9aa60d8eb6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 84.7 MB/s eta 0:00:00
  Created wheel for ecoscape-connectivity: filename=ecoscape_connectivity-0.0.2-py3-none-any.whl size=11054 sha256=435e68da6e24be70e8c63bf3f109c781e8697e5a4922cdbc35b48056e70059c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ffkg97lk/wheels/72/73/2c/f3b584a638e8a6a37479ac7d8823cb5a88a277b40f8c2732f4
Successfully built ecoscape-connectivity
  Cloning https://github.com

In [4]:
# Connecting to Drive.
if not IS_LOCAL:
    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [5]:
import time
from ecoscape_utilities import BirdRun
from ecoscape_connectivity import compute_connectivity
from ecoscape_connectivity.util import read_transmission_csv

## Bird Run Definition

In [6]:
bird_run = BirdRun(DATA_PATH)

birds = []

NUM_SIMULATIONS = 400
BIRD_RUN = "Timing"

birds.append(bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    run_name=BIRD_RUN,
    do_validation=False,
    hop_distance=2, num_spreads=20,
    num_simulations=NUM_SIMULATIONS))

birds.append(bird_run.get_bird_run(
    "acowoo", "Acorn Woodpecker",
    run_name=BIRD_RUN,
    do_validation=False,
    hop_distance=3, num_spreads=15,
    num_simulations=NUM_SIMULATIONS))

birds.append(bird_run.get_bird_run(
    "stejay", "Steller's Jay",
    run_name=BIRD_RUN,
    do_validation=False,
    hop_distance=2, num_spreads=3,
    num_simulations=NUM_SIMULATIONS))

birds.append(bird_run.get_bird_run(
    "stejay", "Steller's Jay",
    run_name=BIRD_RUN,
    do_validation=False,
    hop_distance=1, num_spreads=6,
    num_simulations=NUM_SIMULATIONS))


You need to run the cell below _twice_ to get accurate timings, because the first time it is run, time will be wasted connecting to the GPU.

In [8]:
# Let's do the bird runs.
t = time.time()
for bird in birds:

    # Creates output folder, if missing.
    bird_run.createdir_for_file(bird.repopulation_fn)
    bird_run.createdir_for_file(bird.gradient_fn)

    transmission_d = read_transmission_csv(bird.transmission_fn)

    compute_connectivity(
        habitat_fn=bird.habitat_fn,
        terrain_fn=bird.terrain_fn,
        connectivity_fn=bird.repopulation_fn,
        flow_fn=None,
        permeability_dict=transmission_d,
        gap_crossing=bird.hop_distance,
        num_gaps=bird.num_spreads,
        num_simulations=bird.num_simulations,
        seed_density=4,
        single_tile=True,
    )

    print("Processed", bird.name, "gradient=False", "gap(px):", bird.hop_distance, "num_spreads:", bird.num_spreads, "simulations:", bird.num_simulations)
    print("Time:", time.time() - t)
    t = time.time()

    compute_connectivity(
        habitat_fn=bird.habitat_fn,
        terrain_fn=bird.terrain_fn,
        connectivity_fn=bird.repopulation_fn,
        flow_fn=bird.gradient_fn,
        permeability_dict=transmission_d,
        gap_crossing=bird.hop_distance,
        num_gaps=bird.num_spreads,
        num_simulations=bird.num_simulations,
        seed_density=4,
        single_tile=False,
        tile_size=2048,
        tile_border=256,
    )

    print("Processed", bird.name, "gradient=True", "gap(px):", bird.hop_distance, "num_spreads:", bird.num_spreads, "simulations:", bird.num_simulations)
    print("Time:", time.time() - t)
    t = time.time()

Processed Acorn Woodpecker gradient=False gap(px): 2 num_spreads: 20 simulations: 400
Time: 16.057485818862915


Processed Acorn Woodpecker gradient=True gap(px): 2 num_spreads: 20 simulations: 400
Time: 57.64992642402649


Processed Acorn Woodpecker gradient=False gap(px): 3 num_spreads: 15 simulations: 400
Time: 15.723552227020264


Processed Acorn Woodpecker gradient=True gap(px): 3 num_spreads: 15 simulations: 400
Time: 50.26489591598511


Processed Steller's Jay gradient=False gap(px): 2 num_spreads: 3 simulations: 400
Time: 4.7080042362213135


Processed Steller's Jay gradient=True gap(px): 2 num_spreads: 3 simulations: 400
Time: 11.184788942337036


Processed Steller's Jay gradient=False gap(px): 1 num_spreads: 6 simulations: 400
Time: 5.9216485023498535
Processed Steller's Jay gradient=True gap(px): 1 num_spreads: 6 simulations: 400
Time: 17.6061749458313
